In [1]:
import os
import sys
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader,WebBaseLoader,UnstructuredURLLoader
from langchain_core.runnables import RunnablePassthrough

c:\ConferenceBot\ConferenceBot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
_ = load_dotenv(find_dotenv())  # read local .env file
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [3]:
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

In [4]:
# loading data 
loader = UnstructuredURLLoader(urls=["https://aziz-ashfak.github.io/profile/"])
data = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)


In [6]:
texts = [doc.page_content for doc in docs]

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b85d9a97-cb99-4300-955d-c5577264222a)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./config_sentence_transformers.json
Retrying in 1s [Retry 1/5].


In [8]:
vectorstore = FAISS.from_texts(texts, embeddings)

In [9]:
retrive = vectorstore.as_retriever()

In [10]:
retrive.invoke("Who is Aziz Ashfak?")

[Document(id='aa36c71e-6764-42a5-a3ca-cb82d22e03a0', metadata={}, page_content='ML / AI Engineer · Researcher\n\nBuilding reliable AI systems for real‑world impact.\n\nI specialize in LLMs, RAG systems, and computer vision, grounded in a strong statistics foundation. I build clean, scalable, production-ready AI systems and research tools that deliver real impact\n\nView projects ↓ Hire / collaborate Download resume\n\nNoakhali, Bangladesh · B.Sc. in Statistics (NSTU, 2022–2026)\n\nPhone: 01730644634\n\nEmail: azizashfak@gmail.com\n\nGitHub ↗ LinkedIn ↗ Kaggle ↗ LeetCode ↗\n\nPortrait of Aziz Ashfak\n\nAziz Ashfak\n\nML/AI Engineer · LLMs · RAG · CV\n\nOpen to roles & collaborations\n\n100+\n\nStudents mentored\n\n10+\n\nDeployed apps\n\nRAG systems\n\nCV pipelines\n\nExperience\n\nAI Engineer · ROYALX LLC\n\nRemote · Dhaka, Bangladesh · Aug 2025 – Nov 2025 · Part-time\n\nLLM / RAG\n\nBuilt LLM apps (OpenAI, Groq, HF) with LangChain/LlamaIndex for generation, summarization, and chatbots

In [11]:
prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant that helps people find information about Aziz Ashfak from his personal website.
    "Use the following pieces of context to answer the question at the end. "
    "If you don't know the answer, just say that you don't know, don't try to make up an answer.
    
    context:
    {context}
    
    Question: 
    {question}"""
)

In [12]:
# Convert Documents → string
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [13]:
# Chain
chain = (
    {
        "context": retrive | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)



In [15]:
chain.invoke( "Who is Aziz Ashfak?").content

'Aziz Ashfak is an ML/AI Engineer and Researcher who specializes in Large Language Models (LLMs), RAG systems, and computer vision, with a strong foundation in statistics. He builds reliable AI systems for real-world impact and is open to roles and collaborations in his areas of expertise.'